In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.nn as nn
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import csv
import os
from tensorboardX import SummaryWriter
from sklearn.metrics import roc_auc_score, accuracy_score

from tqdm.notebook import tqdm

from data.datasets import DeClareDataset
from model.deClare import DeClareModel
from metrics.evaluation import DeClareEvaluation

In [2]:
torch.cuda.is_available()

False

# Load Datasets

In [3]:
SNOPES_LOC = "./Datasets/Snopes/snopes.tsv"
#consists of rumors analyzed on the Snopes website along with their credibility labels (true or false), 
#sets of reporting articles, and their respective web sources

POLITIFACT_LOC = "./Datasets/PolitiFact/politifact.tsv"

glove_data_file = "./Glove/glove.6B.100d.txt"

In [4]:
snopes = DeClareDataset(SNOPES_LOC, glove_data_file)

Successfully read news data from ./Datasets/Snopes/snopes.tsv
Number of articles = 29242
Number of claims = 4341
Using pre-built vocabulary


In [5]:
snopes.news_df.head()

,Credibility,Claim_Source,Claim,Article,Article_Source
0,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,there are several holidays throughout that tim...,www.godlikeproductions.com
1,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,defenses against same photographs show images ...,www.sjpba.net
2,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,the best that life could think out we extended...,www.englisher.net
3,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,this entry november 19 2006 published 9 years ...,rss2.com
4,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,place he will not do either said snape the ord...,www.englisher.net


## Preliminary Analysis

In [6]:
batch_size = 64
nb_lstm_units = 64
random_seed = 42

val_split = 0.1
test_split = 0.1
shuffle_dataset = False

In [7]:
# Creating data indices for training and validation splits:
dataset_size = len(snopes)
indices = list(range(dataset_size))

val_split = int(np.floor(val_split * dataset_size))
test_split = val_split + int(np.floor(test_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices, test_indices = indices[test_split:], indices[:val_split], indices[val_split:test_split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [8]:
train_dataloader = DataLoader(snopes, batch_size, sampler=train_sampler)
val_dataloader = DataLoader(snopes, len(val_indices), sampler=val_sampler)
test_dataloader = DataLoader(snopes, len(test_indices), sampler=test_sampler)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
declare = DeClareModel(snopes.initial_embeddings, snopes.claim_source_vocab_size, 
                       snopes.article_source_vocab_size, nb_lstm_units, device)

declare.to(device)

DeClareModel(
  (word_embeddings): Embedding(56755, 100)
  (claim_source_embeddings): Embedding(4342, 4)
  (article_source_embeddings): Embedding(12237, 8)
  (attention_dense): Linear(in_features=200, out_features=1, bias=True)
  (attention_dropout): Dropout(p=0.5, inplace=False)
  (dense_1): Linear(in_features=140, out_features=64, bias=True)
  (dense_1_dropout): Dropout(p=0.5, inplace=False)
  (dense_2): Linear(in_features=64, out_features=64, bias=True)
  (dense_2_dropout): Dropout(p=0.5, inplace=False)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
  (biLSTM): LSTM(100, 64, batch_first=True, bidirectional=True)
)

In [11]:
optimizer = optim.Adam(declare.parameters(), lr=0.005)
BCEloss = nn.BCELoss()

In [12]:
losses = {}
num_epochs = 20
reg_lambda = 0.0002

writer = SummaryWriter()

for epoch in range(num_epochs):
    losses[epoch] = []
    for data_sample in tqdm(train_dataloader):
        
        declare.zero_grad()
        idx = np.argsort(-data_sample[3])

        for i in range(len(data_sample)):
            data_sample[i] = data_sample[i][idx].to(device)

        out = declare(data_sample[0], data_sample[1], data_sample[2], data_sample[3], data_sample[4], data_sample[5])
        loss = BCEloss(out, data_sample[6].float())

        l2_reg = None
        for param in declare.named_parameters():
            if 'dense' in param[0] and 'weight' in param[0]:
                if l2_reg is None:
                    l2_reg = param[1].norm(2)
                else:
                    l2_reg = l2_reg + param[1].norm(2)

        total_loss = loss + reg_lambda*l2_reg

        total_loss.backward()
        optimizer.step()

        writer.add_scalar('total loss', total_loss, epoch)
        writer.add_scalar('regularization loss', l2_reg, epoch)
        writer.add_scalar('loss', loss, epoch)
        writer.add_scalar('pad embedding', declare.word_embeddings(torch.tensor([0]).to(device)).data.mean())
        
        losses[epoch].append(total_loss.data)

Successfully read news data from ./Datasets/Snopes/snopes.tsv
Number of articles = 29242
Number of claims = 4341
Using pre-built vocabulary

  1%|          | 2/366 [00:04<13:56,  2.30s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-6b3cdf890916>", line 9, in <module>
    for data_sample in tqdm(train_dataloader):
  File "C:\Python39\lib\site-packages\tqdm\notebook.py", line 239, in __init__
    self.container = self.status_printer(self.fp, total, self.desc, self.ncols)
  File "C:\Python39\lib\site-packages\tqdm\notebook.py", line 112, in status_printer
    raise ImportError(
ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Durin

In [ ]:
plt.plot(losses[0])

In [ ]:
# torch.save(declare.state_dict(), 'demo_model')
# declare.load_state_dict(torch.load('demo_model'))

In [ ]:
snopes_eval = DeClareEvaluation(declare, test_dataloader, device)
labels, preds = snopes_eval.claim_wise_accuracies()

In [ ]:
true_claim_indices = np.where(labels==1)
false_claim_indices = np.where(labels==0)

In [ ]:
accuracy_score(labels[true_claim_indices], preds[true_claim_indices]>0.5)

In [ ]:
accuracy_score(labels[false_claim_indices], preds[false_claim_indices]>0.5)

In [ ]:
roc_auc_score(labels, preds)

In [ ]:
potifact_df = pd.read_csv(POLITIFACT_LOC, sep='\t', header=None)

In [ ]:
potifact_df.head()

In [ ]:
snopes_df = pd.read_csv(SNOPES_LOC, sep='\t', header=None)
snopes_df.head()